# Julia PyPlot Notebook Settings
How to generate high quality plots, documents, and slides within Julia notebooks using PyPlot and export the notebooks to different file formats.

In [ ]:
using PyPlot
using Random
using TiffImages

## Exporting to pdf, html, docx (Microsoft Word 2007?) and odt (LibreOffice) formats
VS Code has an export to PDF or HTML document formats (see the ... entry after Outline in the top menu bar of the note).  Unfortunately for PDFs, there appears to be no way modify the code/verbatim font size and it is ridiculously large. I tried a number of suggestions from copilot and claude and none of them

- copilot suggestion

`jupyter.exe nbconvert --to pdf --CSSHTMLHeaderPreprocessor.enabled=true --CSSHTMLHeaderPreprocessor.style="custom.css" --output ".\Julia PyPlot Notebook Settings nbconvert.pdf" ".\Julia PyPlot Notebook Settings.ipynb"`

- claude suggestion

`jupyter.exe nbconvert --to pdf --template classic --TemplateExporter.extra_template_basedirs=. --TemplateExporter.extra_template_paths=. --output ".\Julia PyPlot Notebook Settings nbconvert.pdf" ".\Julia PyPlot Notebook Settings.ipynb"`

- using pandoc to generate odt (LibreOffice) files use the command below
  - modify the custom-reference.odt as required
  - code blocks use the Preformatted Text paragraph style
  - verbatim markdown uses the Source_Text character style

`pandoc --from=ipynb --to=odt --reference-doc=custom-reference.odt -o --filter filter-slide-only.py "Julia PyPlot Notebook Settings pandoc.odt" "Julia PyPlot Notebook Settings.ipynb"`

- using pandoc to generate docx
  - again, modify the custom-reference.docx as required
  - code blocks use the Preformatted Text paragraph style but the style values are not taken from the custom-reference.docx file, they also use the Verbatim Char character style which is taken from the custom-reference.docx

`pandoc --from=ipynb --to=docx --reference-doc=custom-reference.docx -o --filter filter-slide-only.py "Julia PyPlot Notebook Settings pandoc.docx" "Julia PyPlot Notebook Settings.ipynb"`

- using pandoc to generate PDFs

## Creating Slides Using nbconvert
Using jupyter-nbconvert relies on at least 2 config files to define the slide appearance and function
* ~/.jupyter/custom/custom.css
* ~/.jupyter/jupyter_nbconvert_config.py
the command line can also overwrite the settings in jupyter_nbconvert_config.py.

A typical command line is

`jupyter nbconvert '.\Julia PyPlot Notebook Settings.ipynb' --to slides --no-input --TagRemovePreprocessor.remove_input_tags='{"document", "hide-code"}' --post serve --SlidesExporter.reveal_width=3840 --SlidesExporter.reveal_height=2160`

where the width and height is that of the display to be used for the presentation.

## Managing The Text And Graphics Image Size
Starting with display resolution and size, my desktop has 23.8" monitors with 4K (3840 x 2160) resolution (16x9 aspect ratio).  The Dell laptop has a 15. (3456x2160).

In [ ]:
display_rows = 3840
display_cols = 2160
display_diagonal_inches = 23.8
desktop_ppi = sqrt(display_rows^2 + display_cols^2) / display_diagonal_inches

## Modify the default plot settings
The following code block makes for better quality plots than the default and allow for dark/light selection.

In [ ]:
# Dark or light mode settings, is there a better way to do this?  Probably Matplotlib style sheets
dark_mode = false
if dark_mode
    # plt.style.use("dark_background")
    foreground_color = "FFFF80"
    background_color = "gray"
else
    # is this really what I want?
    plt.style.use("default")
    foreground_color = "faffb0"
    background_color = "gray"
end
plt.rc("figure", facecolor=background_color)
plt.rc("text", color=foreground_color)
plt.rc("axes", facecolor=background_color)
plt.rc("axes", edgecolor=foreground_color)
plt.rc("axes", labelcolor=foreground_color)
plt.rc("axes", labelcolor=foreground_color)
plt.rc("xtick", color=foreground_color)
plt.rc("xtick", labelcolor=foreground_color)
plt.rc("ytick", color=foreground_color)
plt.rc("ytick", labelcolor=foreground_color)

# This needs to be set to prevent double plotting in jupyter (lab) notebooks
PyPlot.isjulia_display[] = false

plt.rc("figure", dpi=desktop_ppi)
# For with a single figure and header, this works well for images but leaves too much space for plots
plt.rc("figure", figsize=(16, 8))

# savefig.dpi is what sets the resolution of the displayed image in jupyter notebooks
# What is the minimum dpi that renders fonts and plots well? For crisp fonts, it seems that 600 dpi is required
# What is the minimum dpi that renders full size images well, without artifacts? It seems to be about 600 dpi
plt.rc("savefig", dpi=600)
plt.rc("savefig", bbox="tight")
plt.rc("figure", max_open_warning=false)
plt.rc("font", size=5)
plt.rc("lines", linewidth=0.75)
plt.rc("lines", markersize=3)
# new_color_cycle = ["ff7f0e", "1f77b4", "2ca02c", "d62728", "9467bd", "8c564b", "e377c2", "7f7f7f", "bcbd22", "17becf"]
# plt.rc("axes", prop_cycle=plt.matplotlib.cycler(color=new_color_cycle))
;

## Output from a small array

In [ ]:
randn(3, 4)

## A simple plot

In [ ]:
z1 = randn(50)
z2 = randn(50)
z3 = randn(50)
fig = figure()
plot(z1, "-o", label="z_1")
plot(z2, "-.d", label="z_2")
plot(z3, "--s", label="z_3")
grid(true)
xlabel("x-axis")
ylabel("y-axis")
legend()
title("DPI = $(PyPlot.gcf().dpi)")
display(gcf());

## A simple histogram

In [ ]:
z = randn(1536, 1944)
figure()
hist_bins = collect(range(-5.0, 5.0, length=500))
hist(z[:], bins=hist_bins, color="g")
grid(true)
xlim(extrema(hist_bins))
xlabel("Value")
ylabel("Count")
title("Dark Current Histogram")
display(gcf())

## Display an image using imshow

In [ ]:
figure()
imshow(z, cmap="gray")
xlabel("x-axis")
ylabel("y-axis")
title("Normally distributed random numbers")
colorbar()
display(gcf())

## Display an image using pcolormesh

In [ ]:
x_range = collect(1:size(z, 2))
y_range = collect(1:size(z, 1))
vmin = minimum(z)
vmax = maximum(z)

figure()
pcolormesh(x_range, y_range, z, cmap="gray", vmin=vmin, vmax=vmax, shading="nearest")
gca().invert_yaxis()
axis("image")
grid(true)
xlabel("x")
ylabel("y")
colorbar()
title("Normally distributed random numbers")
display(gcf())

## Image with high frequency info

In [ ]:
hf_image = Float32.(TiffImages.load("bias.tiff"))
x_range = collect(1:size(hf_image, 2))
y_range = collect(1:size(hf_image, 1))
vmin = minimum(hf_image)
vmax = maximum(hf_image)

figure()
pcolormesh(x_range, y_range, hf_image, cmap="gray", vmin=vmin, vmax=vmax, shading="nearest")
gca().invert_yaxis()
axis("image")
# grid(true)
xlabel("column index")
ylabel("row index")
colorbar()
title("Image with high frequency information")
display(gcf())